## Autoencoder convolucional

## Instalando bibliotecas

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Importação da base cifar10
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Input, Dense

2025-01-20 15:16:56.364246: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 15:16:56.369444: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-20 15:16:56.430918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 15:16:58.215264: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Carregamento base de dados

In [3]:
(previsores_treinamento, _), (previsores_teste, _) = cifar10.load_data()

In [4]:
previsores_treinamento.shape, previsores_teste.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

## Preprocessamento da base - normalizacao

In [5]:
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_teste = previsores_teste.astype('float32') / 255

In [6]:
previsores_treinamento.shape, previsores_teste.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

## Ajuste nos dados

In [7]:
# Como essa base de dados apresenta imagens coloridas temos 3 canais e não somente
# um canal como na base MNIST. As dimensão dessas imagens é 32 x 32, portanto
# teremos um total de 3072 características (32 x 32 x 3 canais)
previsores_treinamento = previsores_treinamento.reshape((len(previsores_treinamento), np.prod(previsores_treinamento.shape[1:])))
previsores_teste = previsores_teste.reshape((len(previsores_teste), np.prod(previsores_teste.shape[1:])))

## Criação do Autoencoder

In [8]:
# Criação do autoencoder
autoencoder = Sequential()

# ENCODE
# Estrutura do encoder: 3072 - 1536 - 768 - 1536 - 3072
# Você pode testar com outras arquiteturas também, mas uma dica é dividir as
# dimensões pela metade. Por exemplo, para chegar no valor de 1536 dividimos
# 3072 / 2 e para chegar no valor de 768 dividimos 1532 / 2

# Trabalharemos com a dimensão mínima de 768, ou seja, de 3072 pixels reduziremos para 768
# Como são imagens com mais pixels, se diminuirmos muito a dimensão final o autoencoder
# pode apresentar resultados ruins
# O número de entradas é 3072 que equivale ao número total de pixels da imagem
autoencoder.add(Dense(units = 1536, activation = 'relu', input_dim = 3072))
autoencoder.add(Dense(units = 768, activation='relu'))

# DECODE
# Faz o processo inverso, ou seja, 768 - 1536 - 3072
autoencoder.add(Dense(units = 1536, activation='relu'))
autoencoder.add(Dense(units = 3072, activation='sigmoid'))
autoencoder.summary()

/home/lucas/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1536)           │     4,720,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 768)            │     1,180,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1536)           │     1,181,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3072)           │     4,721,664 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,803,392 (45.03 MB)

 Trainable params: 11,803,392 (45.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Compilação e aprendizagem, mantendo os parâmetros padrões
# Os resultados não serão tão bons caso use um número pequeno de épocas
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
autoencoder.fit(previsores_treinamento, previsores_treinamento,
                epochs=100, batch_size=256, validation_data=(previsores_teste, previsores_teste))

2025-01-20 15:17:06.865692: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/100


2025-01-20 15:17:08.093065: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 614400000 exceeds 10% of free system memory.


196/196 ━━━━━━━━━━━━━━━━━━━━ 107s 530ms/step - accuracy: 0.0022 - loss: 0.6482 - val_accuracy: 0.0039 - val_loss: 0.6020
Epoch 2/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 105s 538ms/step - accuracy: 0.0037 - loss: 0.5984 - val_accuracy: 0.0077 - val_loss: 0.5885
Epoch 3/100
 41/196 ━━━━━━━━━━━━━━━━━━━━ 1:21 526ms/step - accuracy: 0.0054 - loss: 0.5854

## Visualizar resultados

In [ ]:
# Cria um novo modelo para capturar somente a aprendizagem da rede, que é o codificador
dimensao_original = Input(shape=(3072,))
camada_encoder1 = autoencoder.layers[0]
camada_encoder2 = autoencoder.layers[1]
encoder = Model(dimensao_original, camada_encoder2(camada_encoder1(dimensao_original)))
encoder.summary()

In [ ]:
# Cria as variáveis com as imagens codificadas e decodificadas
imagens_codificadas = encoder.predict(previsores_teste)
imagens_decodificadas = autoencoder.predict(previsores_teste)

In [ ]:
# Visualiza 10 imagens escolhidas aleatoriamente
numero_imagens = 10
imagens_teste = np.random.randint(previsores_teste.shape[0], size=numero_imagens)
plt.figure(figsize=(18, 18))
for i, indice_imagem in enumerate(imagens_teste):
    # Imagem original, nas dimensões 32x32 com 3 canais
    eixo = plt.subplot(10, 10, i + 1)
    plt.imshow(previsores_teste[indice_imagem].reshape(32, 32, 3))
    plt.xticks(())
    plt.yticks(())
    
    # Imagem codificada, nas dimensões 16x16 com 3 canais
    # Se multiplicar os três valores teremos o valor mínimo que é 768 pixels
    eixo = plt.subplot(10, 10, numero_imagens + i + 1)
    plt.imshow(imagens_codificadas[indice_imagem].reshape(16, 16, 3))
    plt.xticks(())
    plt.yticks(())

    # Imagem decodificada, nas dimensões 32x32 com 3 canais
    eixo = plt.subplot(10, 10, 2 * numero_imagens + i + 1)
    plt.imshow(imagens_decodificadas[indice_imagem].reshape(32, 32, 3))
    plt.xticks(())
    plt.yticks(())
plt.show()